In [1]:
\cd ../qenv

In [46]:
\l test.q

"###################################################################################################"
"                                             TEST"
"###################################################################################################"
"==================================================================================================="
"31) .engine.logic.order.New (UNIT) :READY"
`Global function for processing new orders, amending orders and cancelling orders (amending to 0)
"---------------------------------------------------------------------------------------------------"
caseId state dscr                                                                                                    
---------------------------------------------------------------------------------------------------------------------
212    ERROR Place new buy post only limit order at best price, no previous depth or agent orders should update depth
213    ERROR Place new buy post only limit o

In [140]:
s:((!) . flip(
                (`instrument;(`iId`cntTyp`faceValue`mkprice`smul;enlist(0;0;1;1000;1))); 
                (`account;(`aId`avail`bal`lng`srt`ft`rt`wit`time`froz;enlist(0;0;0;(0 1);(0 -1);0;0;0;z;0))); 
                (`inventory;(`aId`side`mm`upnl`ordQty`ordLoss`amt;flip(0 0;-1 1;0 0;0 0;0 0;0 0;10 10))); 
                (`feetier;(`ftId`vol`bal`ref;flip(0 1;0 0;0 0;0 0))); // Update Account
                (`orderbook;(`price`side`qty`hqty`iqty`vqty`time;flip(1000 1001;2#1;2#100;2#100;2#0;2#0;2#z)));
                (`order;(`oId`aId`iId`ivId`side`oqty`price`reduce`dqty`okind`state`offset;flip(1 2;0 0;0 0;((0 1);(0 1));1 1;200 200;1000 1000;11b;1 1;1 1;0 0;0 210)));
                (`risktier;(`rtId`amt`lev;flip(0 1;50000 250000;125 100))) // Update Account
            ));

In [49]:
.model.Inventory . s[`inventory]

aId side ordQty ordVal ordLoss amt iw mm posVal rpnl avgPrice execCost totEnt upnl lev time
-------------------------------------------------------------------------------------------
0   -1   0             0       10     0                                       0            
0   1    0             0       10     0                                       0            


In [44]:
.model.Level . (`price`side`qty`hqty`iqty`vqty`time;flip(1000 1001;2#1;2#100;2#100;2#0;2#0;2#z))

price iId side qty hqty iqty vqty time                   
---------------------------------------------------------
1000      1    100 100  0    0    2020.12.18T22:08:51.278
1001      1    100 100  0    0    2020.12.18T22:08:51.278


In [141]:
.engine.testutils.SwitchSetupModels[s];

In [96]:
.engine.model.orderbook.Orderbook

price| iId side qty hqty iqty vqty time                   
-----| ---------------------------------------------------
1000 |     1    100 100  0    0    2020.12.18T22:10:21.721
1001 |     1    100 100  0    0    2020.12.18T22:10:21.721


In [114]:
.model.Order . s[`order]

oId aId iId ivId price lprice side sprice trig tif okind oskind reduce state oqty dqty lqty offset einst time
-------------------------------------------------------------------------------------------------------------
1   0   0   1    1000         1                    1            1      0     200  1                          
2   0   0   1    1000         1                    1            1      0     200  1                          


In [142]:
l:0!?[`.engine.modtel.orderbook.Orderbook;(
        (in;`side;x`side);
        (>;(+;`qty;(+;`hqty;(+;`iqty;`vqty)));0);
        (|;(<;(+\;`qty);sum[x`oqty]);
        (=;`i;(*:;`i))));0b;()]

In [143]:
l

price iId side qty hqty iqty vqty time                   
---------------------------------------------------------
1000      1    100 100  0    0    2020.12.18T22:10:21.721
1001      1    100 100  0    0    2020.12.18T22:10:21.721


In [74]:
aqty:sum[l[`qty`iqty`hqty`vqty]]
t

In [144]:
rp:min[(sum x`oqty;first[aqty])]^((thresh-prev[thresh])-(thresh-sum x`oqty));

In [145]:
l[`rp]:rp

In [146]:
o:?[`.engine.model.order.Order;(
            (=;`okind;1);
            (in;`price;l[`price] where (l[`rp]>0));
            (in;`state;(0 1));(>;`oqty;0));0b;()];

In [147]:
l[`price] where (l[`rp]>0)

1000 1001


In [148]:
.engine.model.order.Order

oId| aId iId ivId price lprice side sprice trig tif okind oskind reduce state oqty dqty lqty offset einst time
---| ---------------------------------------------------------------------------------------------------------
0  | 0   0   1    1000         1                                 1            1    1                          
1  | 0   0   1    1000         1                    1            1      0     200  1         0                
2  | 0   0   1    1000         1                    1            1      0     200  1         210              


In [149]:
l

price iId side qty hqty iqty vqty time                    rp 
-------------------------------------------------------------
1000      1    100 100  0    0    2020.12.18T22:10:21.721 200
1001      1    100 100  0    0    2020.12.18T22:10:21.721 400


In [150]:
s:0!((`price`side xkey l) ij (`price`side xgroup o))

In [151]:
pdcl!s pdcl

oId   | 1 2    
side  | 1      
price | 1000   
offset| 0 210  
oqty  | 200 200
lqty  |        
reduce| 11b    
aId   | 0 0    
state | 0 0    


In [152]:
msk:raze[.util.PadM[{x#1}'[count'[s`oId]]]];

In [153]:
pdcl:`oId`side`price`offset`oqty`lqty`reduce`aId`state;
(s pdcl):.util.PadM'[s pdcl];

In [169]:
s`dqty

1 1


In [154]:
// Useful counts 
maxN:max count'[s`offset];
tmaxN:til maxN;
numLvls:count[s`offset];

In [157]:
// TODO new display qty
// Calculate new shifts and max shifts
shft:sum[s`offset`oqty]; // the sum of the order offsets and lqty
mxshft:{$[x>1;max[y];x=1;y;0]}'[maxN;shft]; // the max shft for each price

In [173]:
            nhqty:          .util.Clip[(-/)s`hqty`rp]; // Derive the new hidden qty
            nlqty:          .util.Clip[{?[x>z;(y+z)-x;y]}'[s`rp;s`lqty;s`offset]]; // TODO faster/fix
            ndqty:          .util.Clip[{?[((x<y) and (y>0));x;y]}'[s[`dqty];nlqty]]; // TODO faster/fix
            niqty:          sum'[nlqty-ndqty]; // The new order invisible qty = lqty - display qty
            displaydlt:     (ndqty-s[`dqty]); // Derive the change in the order display qty
            lqtydlt:        (nlqty-s[`lqty]); // Derive the change in the order lqty
            hqtydlt:        (nhqty-s[`hqty]); // Derive the change in hidden order qty
            noffset:        .util.Clip[((-/)s`offset`rp)]; // Subtract the replaced amount and clip<0 (offset includes hqty)
            nqty:           .util.Clip[((-/)s`qty`rp)-(sum'[lqtydlt]+hqtydlt)]; // qty -rp - qty attributed to other qtys
            nvqty:          nqty+sum'[ndqty]; // Derive the new visible qty as order display qty + the new qty from above
            nshft:          nlqty+noffset; // 
            nmxshft:        {$[x>1;max[y];x=1;y;0]}'[maxN;nshft]; // the max shft for each price TODO make faster
            nfilled:        s[`oqty] - nlqty; // New amount that is filled (in total)
            accdlts:        s[`lqty] - nlqty; // The new amounts that will attribute to fills.
            nobupd:         count[nhqty];

In [175]:
// Derived the boolean representation of partially and 
// fully filled orders within the matrix of orders referenced
// above. They should not overlap.f
// get fully filled and then set all others that dont conform to 
// partially filled, by simple exclusion conditional.
nstatus:1*((s[`offset]<=s[`rp])and(nshft<=s[`rp])); // todo mask
nstatus+:2*((sums[s[`offset]]<=s[`rp])and not nstatus); // todo mask

In [176]:
noqty

0 200


In [177]:
// Derive the non agent qty's from the s such that quantities
// such as the visible resultant trades can be derived etc.
notAgentQty: .util.PadM[raze'[flip(
        0^s[`hqty]; // hidden qty
        0^(s[`offset][;0] - 0^s[`hqty]); // first offset
        .util.Clip[0^s[`offset][;1_(tmaxN)] - 0^shft[;-1_(tmaxN)]]; // middle offset + shft
        .util.Clip[s[`vqty]-mxshft] // last qty - maximum shift
    )]];

// Derive the splt which is the combination of the lqty of all orders at 
// a level with the interspaced display qty etc. which is used to derive the
// disparate quantities of trades that occur at a given price level.
splt:{$[count[x];1_(raze raze'[(2#0),(0^x);y]);y]}'[s`lqty;notAgentQty];

// Derives the non-zero trade qtys that occur as a result of the replaced amount 
// returns the quantities by price level.
tqty:{s:sums[y];q:.util.Clip[?[(x-s)>=0;y;x-(s-y)]];q where[q>0]}'[s`rp;splt]; 
numtds:count[raze[tqty]];
numtdslvl:count'[tqty];

In [187]:
o:raze'[(s`oId;s`price;noffset;noqty;ndqty;nstatus)][;where[msk]];
.engine.model.order.Update[flip `oId`price`offset`oqty`dqty`state!o];

In [195]:
fills flip `oId`price`offset`oqty`dqty`state!o

oId price offset oqty dqty state
--------------------------------
1   1000  0      0    0    1    
2   1000  10     200  1    0    
